In [86]:
import numpy as np
from CharVectorizer import CharVectorizer
import pandas as pd
from pandas import Series, DataFrame

import scipy
from scipy.stats import spearmanr

import matplotlib.pyplot as plt
from pylab import rcParams
import seaborn as sb

import sklearn
from sklearn.preprocessing import scale
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn import preprocessing
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
from keras.datasets import imdb


In [87]:
train_data = 'embed_train.csv'
test_data = 'embed_test.csv'
train_df = pd.read_csv(train_data)
test_df = pd.read_csv(test_data)
vectorizer = CharVectorizer("abcdefghijklmnopqrstuvwxyz1234567890")
train_df.columns  = ['firstname','lastname','dob','password',
                'firstname_similarity','lastname_similarity',
                'dob_similarity','aggregate_similarity','score']
test_df.columns  = ['firstname','lastname','dob','password',
                'firstname_similarity','lastname_similarity',
                'dob_similarity','aggregate_similarity','score']
#train_df.head() #if you want to see the shape of the data, uncomment this line


In [88]:
#create encoding function that we will use as a lambda 
#in the next following cells
def encode(cell_entry,charvectorizer):
    cells = [cell_entry]
    maxlen = max(len(cell) for cell in cells)
    matrix = charvectorizer.transform(cells,maxlen)
    return matrix
    
    

In [89]:
#test_df['firstname'].values is the accessor for the firstname column
#fname_vector = (encode(val,vectorizer) for val in train_df['firstname'].values)
#print(train_df['firstname'])
#print(list(fname_vector))
for i in range(len(train_df['firstname'])): #https://www.dataquest.io/blog/settingwithcopywarning/ 
    train_df['firstname'][i] = encode(train_df['firstname'][i],vectorizer)
    train_df['password'][i] = encode(train_df['password'][i],vectorizer)

C:\Users\Axiom\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Axiom\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [90]:
train_df.head()

,firstname,lastname,dob,password,firstname_similarity,lastname_similarity,dob_similarity,aggregate_similarity,score
0,"[[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",McConnell,0000-00-00,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",18.182,13.333,0.000,10.505,1
1,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,...",McConnell,0000-00-00,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",18.182,25.000,0.000,14.394,1
2,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,...",Kragh,3/2/1977,"[[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,...",0.000,18.182,12.500,10.227,1
3,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,...",Marques,6/13/1978,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,...",15.385,12.500,31.579,19.821,1
4,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,...",KTEST,1/10/1969,"[[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,...",0.000,0.000,0.000,0,1


In [118]:
embedding_dim = 6
embeddings = 1 * np.random.randn(len(train_df['firstname']),embedding_dim)  # This will be the embedding matrix
print(train_df.ix[:(0)].values)

[[array([[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
  'McConnell' '0000-00-00'
  array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 

C:\Users\Axiom\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until


In [114]:
#embeddings these weights represent a 1 or a zero
print(embeddings[1:3])
print(train_df['firstname'][1:3])

[[-1.23718864 -1.01585029  0.43000853 -0.23621366  0.78981923  1.72737948]
 [ 0.11113641 -0.13857883  0.33721756 -0.91252538 -0.9679848   0.4985273 ]]
1    [[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,...
2    [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,...
Name: firstname, dtype: object
